In [1]:
from dotenv import load_dotenv
import os
from supabase import create_client

load_dotenv()

SUPABASE_URL = os.environ["SUPABASE_URL"]
SUPABASE_KEY = os.environ["SUPABASE_ANON_KEY"]  # o SUPABASE_SERVICE_ROLE_KEY si aplica

supabase = create_client(SUPABASE_URL, SUPABASE_KEY)

In [2]:
from openai import OpenAI
client = OpenAI(api_key=os.environ["OPENAI_API_KEY"])

def embed(text: str) -> list[float]:
    r = client.embeddings.create(model="text-embedding-3-small", input=text)
    return r.data[0].embedding

def to_pgvector(v: list[float]) -> str:
    return "[" + ",".join(f"{x:.8f}" for x in v) + "]"

In [3]:
query = "what is key when preparing students"
q_emb = to_pgvector(embed(query))

res = supabase.rpc("match_documents", {
    "query_embedding": q_emb,
    "match_count": 5,
    "filter": {},  # ej: {"source": "docs"} si guardaste metadata así
}).execute()

res.data

[{'context': 'critical thinking—that will help them stand out to future employers.\nIn the face of this crisis, professors need to rethink how schools \napproach teaching, both in terms of the skills we cultivate and the \nattitudes we encourage. If we don’t, our students may face both  \na decline in their self-esteem and unfulfilled goals for their \nfuture careers.\nOver the past few years, I’ve incorporated several exercises and \nassignments into my courses that help students master AI use  \nwhile also recognizing what value they can bring to an AI world.  \nHere are five that you can adopt for your own classroom.\n1. Explain large language models through the  \n“say the same thing” game\nEarly in a course—usually during the first class—I do an exercise  \nthat demonstrates how large language models (LLMs), the technology \nunderlying gen AI, work. Students are paired off and instructed',
  'metadata': {'loc': {'lines': {'to': 1401, 'from': 1387}},
   'pdf': {'info': {'Title': 'H